# app

> Running the system

In [ ]:
#| default_exp app

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Dict, List, Callable, Optional, Any, Union
from triggerkit import util, snowflake, jobs, actions

In [ ]:
#| export

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('snowflake_actions.main')

In [ ]:
#| exporti

def _parse_arguments(args=None):
    """Parse command line arguments.
    
    Args:
        args: Optional list of command line arguments. If None, will use sys.argv[1:].
        
    Returns:
        Parsed arguments namespace
    """
    parser = argparse.ArgumentParser(description='Snowflake Action System')
    
    parser.add_argument(
        '-c', '--config',
        default=os.environ.get('CONFIG_PATH', 'util.toml'),
        help='Path to TOML configuration file (default: util.toml or CONFIG_PATH env var)'
    )
    
    parser.add_argument(
        '--list-actions',
        action='store_true',
        help='List available actions and exit'
    )
    
    parser.add_argument(
        '--check-config',
        action='store_true',
        help='Check configuration file syntax and exit'
    )
    
    parser.add_argument(
        '-v', '--verbose',
        action='store_true',
        help='Enable verbose logging'
    )
    
    return parser.parse_args(args)

In [ ]:
#| exporti

def init(config_path: str):
    """
    Initialize the application.
    
    **Args:**
        \n • `config_path`: Path to TOML config file
        
    **Returns:**
        None
    """
    # Load configuration
    config = util.load_config(config_path)
    util.logger.info(f"Successfully loaded config from {config_path}")
    util.config = config
    util.SNOWFLAKE_CONFIG = config['snowflake']
    util.logger.info(f"Successfully set util.SNOWFLAKE_CONFIG")

    # Connect to Snowflake
    snowflake_config = config.get('snowflake', {})
    util.snowflake_connection = snowflake.connect(snowflake_config)
    
    # Register Views
    snowflake.init_view_registry(config)

    # Schedule Check Schema Jobs
    if config['snowflake'].get('check_schema') and config['snowflake'].get('schema_check_freq'):
        # General Job to add new views from schema
        config['jobs'].append({
            'name': 'Check Schema For Views',
            'view': 'tk_check_schema',
            'actions': 'Get DDL and Register Views',
            'schedule': config['snowflake']['schema_check_freq']
        })

        # Job to check schema for views with job configurations and create the jobs
        config['jobs'].append({
            'name': 'Check Schema For Configured Views',
            'view': 'tk_cofigured_views',
            'actions': ['Get DDL and Register Views','Create Job From View'],
            'schedule': config['snowflake']['schema_check_freq']
        })
    
    # Store config for other components to access
    util.config = config
    
    # Set up email and Slack configurations for actions to use
    if 'email' in config:
        global EMAIL_CONFIG
        EMAIL_CONFIG = config['email']
    
    if 'slack' in config:
        global SLACK_CONFIG
        SLACK_CONFIG = config['slack']
    
    # Schedule jobs
    jobs.schedule_jobs(config)
    
    # Run the scheduler
    jobs.run_scheduler()

In [ ]:
#| export

def main(args=None):
    """Main entry point for the application.
    
    **Args:**
        \n • `args`: Optional list of command line argument strings, or parsed args object.
        \n \t    If None, will parse from command line.
    """
    if args is None or not isinstance(args, argparse.Namespace):
        args = _parse_arguments(args)
    
    # Set log level based on verbosity
    if args.verbose:
        logging.getLogger().setLevel(logging.DEBUG)
        logger.debug("Verbose logging enabled")
    
    # Check if config file exists
    config_path = Path(args.config)
    if not config_path.exists():
        logger.error(f"Configuration file not found: {config_path}")
        sys.exit(1)
    
    logger.info(f"Using configuration file: {config_path}")
    
    # Just list actions if requested
    if args.list_actions:
        actions.list_available_actions()
        sys.exit(0)
    
    # Just check config if requested
    if args.check_config:
        try:
            config = util.load_config(args.config)
            logger.info(f"Configuration file is valid: {args.config}")
            
            # Display summary of configuration
            logger.info(f"Found {len(util.get('views', []))} views")
            logger.info(f"Found {len(util.get('jobs', []))} jobs")
            
            # Display enabled jobs
            enabled_jobs = [j for j in util.get('jobs', []) if j.get('enabled', True)]
            logger.info(f"Enabled jobs ({len(enabled_jobs)}):")
            for job in enabled_jobs:
                logger.info(f"  - {job.get('name')}: {job.get('view')} → {job.get('actions')} ({job.get('schedule')})")
            
            sys.exit(0)
        except Exception as e:
            logger.error(f"Error validating configuration: {str(e)}")
            sys.exit(1)
    
    # Initialize and run the application
    try:
        logger.info("Starting Snowflake Action System")
        init(args.config)
    except KeyboardInterrupt:
        logger.info("Shutting down Snowflake Action System")
    except Exception as e:
        logger.error(f"Application error: {str(e)}", exc_info=True)
        sys.exit(1)

In [ ]:
#| exporti
def _is_running_in_notebook():
    try:
        # This will only work in IPython/Jupyter environments
        from IPython import get_ipython
        if get_ipython() is not None:
            return True
        return False
    except ImportError:
        return False

In [ ]:
#| exporti

if __name__ == "__main__" and not _is_running_in_notebook():
    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()